<a href="https://colab.research.google.com/github/Ramki13084/tensorflow_sample/blob/main/GoogleNet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import math
import numpy as np
import tensorflow as tf
from keras import backend
from tensorflow.keras.layers import Concatenate

In [ ]:
!pip install tf-slim

     |████████████████████████████████| 352 kB 5.0 MB/s 


In [47]:
#parameters
output_class = 10 # no of classes defined as 10. this parameter is customizable.

In [4]:
def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):
  """Utility function to apply conv + BN.
  Args:
   1st input x: input tensor.
   2nd input filters: filters in `Conv2D`.
   3rd input num_row: height of the convolution kernel.
   4th input num_col: width of the convolution kernel.
   5th input  padding: padding mode in `Conv2D`.
   6th input strides: strides in `Conv2D`.
    name: name of the ops; will become `name + '_conv'`
      for the convolution and `name + '_bn'` for the
      batch norm layer.
  Returns:
    Output tensor after applying `Conv2D` and `BatchNormalization`.
  """
  if name is not None:
    bn_name = name + '_bn'
    conv_name = name + '_conv'
  else:
    bn_name = None
    conv_name = None
  if backend.image_data_format() == 'channels_first':
    bn_axis = 1
  else:
    bn_axis = 3
  x = layers.Conv2D(
      filters, (num_row, num_col),
      strides=strides,
      padding=padding,
      use_bias=False,
      name=conv_name)(
          x)
  x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
  x = layers.Activation('relu', name=name)(x)
  return x

In [44]:
x = np.arange(20).reshape(2,2,5)
print(x)

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]]


In [45]:
y = np.arange(20, 40).reshape(2, 2, 5)
print(y)

[[[20 21 22 23 24]
  [25 26 27 28 29]]

 [[30 31 32 33 34]
  [35 36 37 38 39]]]


In [46]:
concat = tf.keras.layers.Concatenate(axis=-1)([x, y])
concat.shape
print(concat)  # IF axis is given -1 then both x and y tensor should have same shape.
# IF axis is given 1 then x and y tensor should have same tensor shape in y axis.

tf.Tensor(
[[[ 0  1  2  3  4 20 21 22 23 24]
  [ 5  6  7  8  9 25 26 27 28 29]]

 [[10 11 12 13 14 30 31 32 33 34]
  [15 16 17 18 19 35 36 37 38 39]]], shape=(2, 2, 10), dtype=int64)


In [8]:
def inception(x, filters):
    # 1x1
    path1 = conv2d_bn(x,filters[0],1,1)

    # 1x1->3x3
    path2 = conv2d_bn(x,filters[1][0],1,1)
    path2 = conv2d_bn(path2,filters[1][1], 3,3)
    
    # 1x1->5x5
    path3 = conv2d_bn(x,filters[2][0],1,1)
    path3 = conv2d_bn(path3,filters[2][1], 5,5)

    # 3x3->1x1
    path4 = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    path4 = conv2d_bn(path4,filters[3],1,1)(path4)

    return Concatenate(axis=-1)([path1,path2,path3,path4])

In [48]:
def auxiliary(x, name=None):
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = conv2d_bn(layer,128,1,1)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=output_class, activation='softmax', name=name)(layer)
    return layer

In [50]:
def googlenet():
    layer_in = Input(shape=IMAGE_SHAPE)
    
    # stage-1
    layer = conv2d_bn(layer_in,64,7,7,padding='same',strides=(2,2))
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    layer = BatchNormalization()(layer)

    # stage-2
    layer = conv2d_bn(layer,64,1,1)
    layer = conv2d_bn(layer,192,3,3)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)

    # stage-3
    layer = inception(layer, [ 64,  (96,128), (16,32), 32]) #3a
    layer = inception(layer, [128, (128,192), (32,96), 64]) #3b
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-4
    layer = inception(layer, [192,  (96,208),  (16,48),  64]) #4a
    aux1  = auxiliary(layer, name='aux1')
    layer = inception(layer, [160, (112,224),  (24,64),  64]) #4b
    layer = inception(layer, [128, (128,256),  (24,64),  64]) #4c
    layer = inception(layer, [112, (144,288),  (32,64),  64]) #4d
    aux2  = auxiliary(layer, name='aux2')
    layer = inception(layer, [256, (160,320), (32,128), 128]) #4e
    layer = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer)
    
    # stage-5
    layer = inception(layer, [256, (160,320), (32,128), 128]) #5a
    layer = inception(layer, [384, (192,384), (48,128), 128]) #5b
    layer = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(layer)
    
    # stage-6
    layer = Flatten()(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=256, activation='linear')(layer)
    main = Dense(units=CLASS_NUM, activation='softmax', name='main')(layer)
    
    model = Model(inputs=layer_in, outputs=[main, aux1, aux2])
    
    return model

In [51]:
def InceptionV3(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'):

        # Determine proper input shape
        input_shape = imagenet_utils.obtain_input_shape(
            input_shape,
            default_size=299,
            min_size=75,
            data_format=backend.image_data_format(),
            require_flatten=include_top,
            weights=weights)

        if input_tensor is None:
          img_input = layers.Input(shape=input_shape)
        else:
          if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
          else:
            img_input = input_tensor

        if backend.image_data_format() == 'channels_first':
          channel_axis = 1
        else:
          channel_axis = 3

        x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid') # (297-3)/2 + 1=147+1 = 148
        x = conv2d_bn(x, 32, 3, 3, padding='valid') # (148-3)/1+1 = 146
        x = conv2d_bn(x, 64, 3, 3) # (146+2-3)/1+1 = 146
        x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x) # 146/2 = 73

        x = conv2d_bn(x, 80, 1, 1, padding='valid') # (73-1)/1+1 = 73
        x = conv2d_bn(x, 192, 3, 3, padding='valid') # (73-3)/1+1=71
        x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x) # (71+2(1)-3)/2 =35

        # mixed 0: 35 x 35 x 256
        branch1x1 = conv2d_bn(x, 64, 1, 1) 

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
        x = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                              axis=channel_axis,
                              name='mixed0')

        # mixed 1: 35 x 35 x 288
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
        x = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                              axis=channel_axis,
                              name='mixed1')

        # mixed 2: 35 x 35 x 288
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
        x = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                              axis=channel_axis,
                              name='mixed2')

        # mixed 3: 17 x 17 x 768
        branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

        branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = layers.concatenate([branch3x3, branch3x3dbl, branch_pool],
                              axis=channel_axis,
                              name='mixed3')

        # mixed 4: 17 x 17 x 768
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 128, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 128, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                              axis=channel_axis,
                              name='mixed4')

        # mixed 5, 6: 17 x 17 x 768
        for i in range(2):
          branch1x1 = conv2d_bn(x, 192, 1, 1)

          branch7x7 = conv2d_bn(x, 160, 1, 1)
          branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
          branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

          branch7x7dbl = conv2d_bn(x, 160, 1, 1)
          branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
          branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
          branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
          branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

          branch_pool = layers.AveragePooling2D((3, 3),
                                                strides=(1, 1),
                                                padding='same')(
                                                    x)
          branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
          x = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                                axis=channel_axis,
                                name='mixed' + str(5 + i))

        # mixed 7: 17 x 17 x 768
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 192, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 192, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = layers.AveragePooling2D(
            (3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                              axis=channel_axis,
                              name='mixed7')

        # mixed 8: 8 x 8 x 1280
        branch3x3 = conv2d_bn(x, 192, 1, 1)
        branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2, 2), padding='valid')

        branch7x7x3 = conv2d_bn(x, 192, 1, 1)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
        branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
        branch7x7x3 = conv2d_bn(
            branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

        branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
        x = layers.concatenate([branch3x3, branch7x7x3, branch_pool],
                              axis=channel_axis,
                              name='mixed8')

        # mixed 9: 8 x 8 x 2048
        for i in range(2):
          branch1x1 = conv2d_bn(x, 320, 1, 1)

          branch3x3 = conv2d_bn(x, 384, 1, 1)
          branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
          branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
          branch3x3 = layers.concatenate([branch3x3_1, branch3x3_2],
                                        axis=channel_axis,
                                        name='mixed9_' + str(i))

          branch3x3dbl = conv2d_bn(x, 448, 1, 1)
          branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
          branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
          branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
          branch3x3dbl = layers.concatenate([branch3x3dbl_1, branch3x3dbl_2],
                                            axis=channel_axis)

          branch_pool = layers.AveragePooling2D((3, 3),
                                                strides=(1, 1),
                                                padding='same')(
                                                    x)
          branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
          x = layers.concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                                axis=channel_axis,
                                name='mixed' + str(9 + i))
        if include_top:
          # Classification block
          x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
          imagenet_utils.validate_activation(classifier_activation, weights)
          x = layers.Dense(classes, activation=classifier_activation,
                          name='predictions')(x)
        else:
          if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
          elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)

        # Ensure that the model takes into account
        # any potential predecessors of `input_tensor`.
        if input_tensor is not None:
          inputs = layer_utils.get_source_inputs(input_tensor)
        else:
          inputs = img_input
        # Create model.
        model = training.Model(inputs, x, name='inception_v3')

        # Load weights.
        if weights == 'imagenet':
          if include_top:
            weights_path = data_utils.get_file(
                'inception_v3_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,
                cache_subdir='models',
                file_hash='9a0d58056eeedaa3f26cb7ebd46da564')
          else:
            weights_path = data_utils.get_file(
                'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,
                cache_subdir='models',
                file_hash='bcbd6486424b2319ff4ef7d526e38f63')
          model.load_weights(weights_path)
        elif weights is not None:
          model.load_weights(weights)

  return model